In [8]:
#3.5.7
# Load the TensorBoard notebook extension
#%load_ext tensorboard
import tensorflow as tf
import datetime
from keras.datasets import reuters

# Clear any logs from previous runs
!rm -rf ./logs/ 

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

print(len(train_data))
print(len(test_data))

word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics='accuracy')

#3.5.4
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

#3.18
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=128,
                    validation_data=(x_val, y_val),
                    callbacks=[tensorboard_callback])

results = model.evaluate(x_test, one_hot_test_labels)


2021-11-12 17:13:36.124290: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-12 17:13:36.124309: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-12 17:13:36.124536: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


8982
2246
Epoch 1/20
15/63 [======>.......................] - ETA: 0s - loss: 2.4184 - accuracy: 0.5000

2021-11-12 17:13:38.205238: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-12 17:13:38.205257: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-12 17:13:38.252865: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-12 17:13:38.254162: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-12 17:13:38.256151: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211112-171336/train/plugins/profile/2021_11_12_17_13_38

2021-11-12 17:13:38.257199: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/20211112-171336/train/plugins/profile/2021_11_12_17_13_38/Roys-MacBook-Pro.local.trace.json.gz
2021-11-12 17:13:38.259699: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211112-171336/tr

63/63 [==============================] - 1s 13ms/step - loss: 1.5459 - accuracy: 0.6665 - val_loss: 1.0951 - val_accuracy: 0.7530
Epoch 2/20
63/63 [==============================] - 1s 10ms/step - loss: 0.7574 - accuracy: 0.8334 - val_loss: 0.9092 - val_accuracy: 0.8080
Epoch 3/20
63/63 [==============================] - 1s 10ms/step - loss: 0.4492 - accuracy: 0.9029 - val_loss: 0.8649 - val_accuracy: 0.8120
Epoch 4/20
63/63 [==============================] - 1s 10ms/step - loss: 0.2969 - accuracy: 0.9349 - val_loss: 0.8578 - val_accuracy: 0.8160
Epoch 5/20
63/63 [==============================] - 1s 10ms/step - loss: 0.2207 - accuracy: 0.9460 - val_loss: 0.9343 - val_accuracy: 0.8100
Epoch 6/20
63/63 [==============================] - 1s 10ms/step - loss: 0.1857 - accuracy: 0.9498 - val_loss: 1.0438 - val_accuracy: 0.7980
Epoch 7/20
63/63 [==============================] - 1s 10ms/step - loss: 0.1664 - accuracy: 0.9531 - val_loss: 0.9875 - val_accuracy: 0.7930
Epoch 8/20
63/63 [======

In [9]:
#3.5.7
print(history.history['accuracy'][-1])
print(history.history['val_accuracy'][-1])

predictions = model.predict(x_test)
print(predictions[0])
print(np.sum(predictions[0]))
print(np.argmax(predictions[0]))

0.9564018845558167
0.7839999794960022
[1.82201862e-08 3.61448969e-07 1.59883879e-13 9.97318208e-01
 2.39316863e-03 5.65139959e-15 1.46463969e-11 5.56240926e-13
 1.21847886e-04 1.24426123e-08 1.03950226e-08 3.87789660e-05
 1.24285882e-09 6.76775517e-06 3.22959853e-11 6.66567019e-12
 3.35920618e-07 1.25483174e-11 7.22735871e-10 1.10547917e-04
 1.83382906e-06 7.90603190e-06 1.85545671e-13 4.19065449e-09
 6.95495788e-12 6.48783693e-10 1.43767776e-16 5.19827881e-14
 2.60034461e-09 1.24631944e-10 5.94198024e-10 5.08277333e-14
 1.19791928e-11 4.25187966e-15 1.04821776e-10 6.00657154e-12
 1.00500820e-07 2.66979082e-12 4.74583983e-11 2.86355887e-11
 1.45243395e-09 8.05551372e-12 1.43332501e-15 1.00334468e-13
 1.11948265e-13 3.38911977e-14]
0.9999999
3
